In [1]:
# !wget https://huggingface.co/huseinzol05/translation-ms-en-small/resolve/main/model.pb

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
import tensorflow as tf

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [5]:
# g = load_graph('model.pb')
# graph_def = g.as_graph_def()
# for node in graph_def.node:
#     n = g.get_tensor_by_name(f'{node.name}:0')
#     if 'encdec_attention/multihead_attention/dot_product_attention' in node.name:
#         print(node.name, n)

In [6]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry


/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# !wget https://huggingface.co/huseinzol05/bpe/resolve/main/ms-en.subwords

import text_encoder

In [8]:
encoder = text_encoder.SubwordTextEncoder('ms-en.subwords')

In [48]:
x = encoder.encode('saya suka ayam sangatttta')
y = encoder.encode('i like chickoon so much')
x, y

([197, 3728, 2744, 18569, 6057, 10436],
 [55, 227, 4311, 3891, 10901, 225, 1009])

In [32]:
encoder._subtoken_ids_to_tokens(x)

['saya', 'suka', 'ayam', 'sangatttta']

In [10]:
@registry.register_problem
class TRANSLATION32k(translate.TranslateProblem):

    @property
    def additional_training_datasets(self):
        """Allow subclasses to add training datasets."""
        return []
    
    def feature_encoders(self, data_dir):
        encoder = text_encoder.SubwordTextEncoder('ms-en.subwords')
        return {'inputs': encoder, 'targets': encoder}

In [11]:
PROBLEM = 'translatio_n32k'
problem = problems.problem(PROBLEM)

In [12]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        self.hparams = hparams
        translate_model = registry.model('transformer')(hparams, Modes.TRAIN)
        self.translate_model = translate_model
        self.out = translate_model(features)

In [13]:
model = Model()

INFO:tensorflow:Setting T2TModel mode to 'train'


INFO:tensorflow:Setting T2TModel mode to 'train'


INFO:tensorflow:Using variable initializer: uniform_unit_scaling


INFO:tensorflow:Using variable initializer: uniform_unit_scaling


INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_25880_512.bottom


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_25880_512.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_25880_512.top


INFO:tensorflow:Transforming body output with symbol_modality_25880_512.top


In [14]:
model.translate_model.attention_weights

{'transformer/body/encoder/layer_0/self_attention/multihead_attention/dot_product_attention': <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/dot_product_attention/logits': <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/dot_product_attention/add:0' shape=(?, 8, ?, ?) dtype=float32>,
 'transformer/body/encoder/layer_1/self_attention/multihead_attention/dot_product_attention': <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_1/self_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 'transformer/body/encoder/layer_1/self_attention/multihead_attention/dot_product_attention/logits': <tf.Tensor 'transformer/parallel_0_4/tr

In [15]:
model.hparams.num_hidden_layers

6

In [19]:
attn = []
for i in range(model.hparams.num_hidden_layers):
    encdec_att = model.translate_model.attention_weights[
      "transformer/body/decoder/layer_%i/encdec_attention/multihead_attention/dot_product_attention" % i]
    attn.append(encdec_att)

In [20]:
attn

[<tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/decoder/layer_0/encdec_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/decoder/layer_1/encdec_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/decoder/layer_2/encdec_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/decoder/layer_3/encdec_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 <tf.Tensor 'transformer/parallel_0_4/transformer/transformer/body/decoder/layer_4/encdec_attention/multihead_attention/dot_product_attention/attention_weights:0' shape=(?, 8, ?, ?) dtype=float32>,
 <tf.Tenso

In [ ]:
model.translate_model.attention_weights['transformer/body/decoder/layer_0/encdec_attention/multihead_attention/dot_product_attention/logits']

In [21]:
sess = tf.InteractiveSession()


2022-06-27 00:21:31.469195: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-06-27 00:21:31.469533: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x58af760 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-27 00:21:31.469545: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-27 00:21:31.470592: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-27 00:21:31.473198: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-27 00:21:31.473213: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-27 00:21:31.473216: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincome

In [22]:
sess.run(tf.global_variables_initializer())

In [49]:
o = sess.run(attn, feed_dict = {
    model.X: [x],
    model.Y: [y]
})

In [50]:
for i in range(len(o)):
    print(i, o[i].shape)

0 (1, 8, 7, 6)
1 (1, 8, 7, 6)
2 (1, 8, 7, 6)
3 (1, 8, 7, 6)
4 (1, 8, 7, 6)
5 (1, 8, 7, 6)


In [51]:
len(x), len(y)

(6, 7)

In [58]:
a = np.mean(o[-1][0], axis =0).T
a.shape

(6, 7)

In [38]:
import numpy as np

In [64]:
np.mean([a[0], a[0]], axis = 0).shape

(7,)

In [100]:
def merged(tokens, weights):
    i = 0
    n_tokens = len(tokens)
    new_paired_tokens = []
    while i < n_tokens:
        current_token = tokens[i]
        current_weight = weights[i]
        if not current_token.endswith('_'):
            merged_token = ''
            merged_weight = []
            while (
                not current_token.endswith('_')
            ):
                merged_token = merged_token + current_token.replace('_', '')
                merged_weight.append(current_weight)
                i = i + 1
                current_token = tokens[i]
                current_weight = weights[i]
            merged_token = merged_token + tokens[i]
            merged_weight.append(weights[i])
            merged_weight = np.mean(merged_weight, axis=0)
            new_paired_tokens.append((merged_token, merged_weight))
            i = i + 1
        else:
            new_paired_tokens.append((current_token, current_weight))
            i = i + 1
    words = [
        i[0].replace('▁', '') for i in new_paired_tokens
    ]
    weights = [i[1] for i in new_paired_tokens]
    return words, np.array(weights)

In [108]:
concatenated = [encoder._subtoken_id_to_subtoken_string(s) for s in x]
c_x, a_x = merged(concatenated, a)

In [109]:
a_x.shape

(4, 7)

In [110]:
concatenated = [encoder._subtoken_id_to_subtoken_string(s) for s in y]
c_y, a_y = merged(concatenated, a_x.T)

In [114]:
a_y.T

array([[0.18293898, 0.20045616, 0.19267084, 0.19390467, 0.20172535],
       [0.12858678, 0.14654504, 0.1563607 , 0.17914012, 0.15539902],
       [0.1781643 , 0.158537  , 0.1996336 , 0.204992  , 0.18645568],
       [0.17010331, 0.1648206 , 0.15044494, 0.1406544 , 0.15213999]],
      dtype=float32)

In [112]:
len(c_x), len(c_y)

(4, 5)